In [ ]:
import numpy as np
import os
import signalflow as sf
from PIL import Image
from IPython.display import Audio, display
from pixasonics.core import App, Mapper
from pixasonics.features import *
from pixasonics.synths import Theremin, Oscillator, FilteredNoise, SimpleFM

In [ ]:
# Create app
app = App(image_size=(800, 800))

In [ ]:
# now use all images in the folder
img_folder = "images/cellular_dataset/single_channel_16bit/"
img_files = os.listdir(img_folder)
imgs_red = [f for f in img_files if f.endswith("w2.TIF")] # only red channel images
imgs_green = [f for f in img_files if f.endswith("w1.TIF")] # only green channel images
imgs_blue = [f for f in img_files if f.endswith("w3.TIF")] # only blue channel images
imgs = []
for img_red, img_green, img_blue in zip(imgs_red, imgs_green, imgs_blue):
    img_path_red = os.path.join(img_folder, img_red)
    img_path_green = os.path.join(img_folder, img_green)
    img_path_blue = os.path.join(img_folder, img_blue)
    img_red = Image.open(img_path_red)
    img_green = Image.open(img_path_green)
    img_blue = Image.open(img_path_blue)
    img_red = np.array(img_red)
    img_green = np.array(img_green)
    img_blue = np.array(img_blue)
    img = np.stack([img_red, img_green, img_blue], axis=-1) # now the last dimension is the channel dimension
    imgs.append(img)
img = np.stack(imgs, axis=-1) # now the last dimension is the layer dimension
print(img.shape)
app.load_image_data(img) # load as numpy array

In [ ]:
from pixasonics.core import AppRegistry

app_registry = AppRegistry()

app_registry._apps

In [ ]:
app2 = App(image_size=(500, 500))
img_path = "images/test.jpg"
app2.load_image_file(img_path)
mean_red2 = MeanChannelValue(filter_channels=0, name="MeanRed")
# attach the feature to the app
app2.attach_feature(mean_red2)

# create a Theremin, a simple sine wave synth that we will use to sonify the mean pixel value
theremin2 = Theremin()
# attach the Theremin to the app
app2.attach_synth(theremin2)

# create a Mapper that will map the mean red pixel value (within the Probe) to the frequency of the Theremin
red2freq2 = Mapper(
    mean_red2, 
    theremin2["frequency"], 
    exponent=2, name="Red2Freq") # cubic mapping curve for a more "linear" feel of frequency changes
# attach the Mapper to the app
app2.attach_mapper(red2freq2)

In [ ]:
app.graph.status

In [ ]:
import signalflow as sf
graph = sf.AudioGraph.get_shared_graph()
print(graph is None)

In [ ]:
import signalflow as sf

graph = None
buf = sf.Buffer(1, 48000)

class TestPatch(sf.Patch):
    def __init__(self):
        super().__init__()
        param = self.add_input("param")

        out = param * sf.SineOscillator(440)
        self.set_output(out)

def create_audio_graph(nrt=False):
    graph = sf.AudioGraph.get_shared_graph()
    output_device = sf.AudioOut_Dummy(2) if nrt else None
    if graph is not None:
        graph.destroy()
    graph = sf.AudioGraph(
        start=True,
        output_device=output_device)
    my_patch = TestPatch()
    my_patch.set_input("param", 0.5)
    graph.play(my_patch)
    if nrt:
        graph.render_to_buffer(buf)
    graph.stop(my_patch)
    return graph

print(graph) # should be None

graph = create_audio_graph(nrt=False)

print("RT", graph.status)

graph = create_audio_graph(nrt=True)

print("NRT", graph.status)

graph = create_audio_graph(nrt=False)

print("RT2", graph.status)

In [ ]:
from pixasonics.core import App, Mapper
from pixasonics.features import MeanChannelValue
from pixasonics.synths import Theremin

app = App(image_size=(800, 800))
app.load_image_file("images/cellular_dataset/merged_8bit/Timepoint_001_220518-ST_C03_s1.jpg")

mean_red = MeanChannelValue(filter_channels=0, name="MeanRed")
app.attach_feature(mean_red)

num_instances = 5

for i in range(num_instances):
    theremin = Theremin()
    app.attach_synth(theremin)

    red2freq = Mapper(mean_red, theremin["frequency"], exponent=2, name=f"Red2Freq{i}")
    app.attach_mapper(red2freq)

    red2amp = Mapper(mean_red, theremin["amplitude"], exponent=1, name=f"Red2Amp{i}")
    app.attach_mapper(red2amp)

    red2pan = Mapper(mean_red, theremin["panning"], exponent=1, name=f"Red2Pan{i}")
    app.attach_mapper(red2pan)

app.interaction_mode = "toggle"
app.audio = True

In [ ]:
import signalflow as sf
graph = sf.AudioGraph.get_shared_graph()
if graph is not None:
    graph.destroy()
graph = sf.AudioGraph(output_device=sf.AudioOut_Dummy(2))

In [ ]:
import signalflow as sf
graph = sf.AudioGraph()

class TestPatch(sf.Patch):
    def __init__(self):
        super().__init__()
        param = self.add_input("param")

        out = param * sf.SineOscillator(440)
        self.set_output(out)

patch = TestPatch()
patch.set_input("param", 0.5)

In [ ]:
patch.set_input("param", 1)

In [ ]:
patch.play()

In [ ]:
patch.stop()

In [ ]:
print(graph.structure)

In [ ]:
graph.status